In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("✅ 成功讀取 OPENAI_API_KEY")
else:
    print("❌ 沒有讀取到 OPENAI_API_KEY，請檢查 .env 檔")

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=1)
response = llm.invoke("hello")
print("✅ LLM 測試回應：", response.content)


✅ 成功讀取 OPENAI_API_KEY


C:\Users\chen\AppData\Local\Temp\ipykernel_34412\2719606916.py:14: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o", temperature=1)


✅ LLM 測試回應： Hello! How can I assist you today?


In [2]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.vectorstores import Chroma
# from qdrant_client import QdrantClient
# from langchain_qdrant import QdrantVectorStore
from langchain_core.runnables.history import RunnableWithMessageHistory
from typing import Dict
from langchain.schema.runnable import RunnableMap, RunnableLambda
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.chat_history import InMemoryChatMessageHistory

# 配置 LLM 模型
generator_llm = ChatOpenAI(
    model_name="gpt-4o",   # 或 "gpt-4", "gpt-3.5-turbo"
    temperature=0.3,       # 控制回答穩定性。推薦值：0~0.3（任務導向），0.7~1（創意任務）
    max_tokens=8192,       # 控制回應長度。推薦根據任務調整，如摘要可短，技術問答可設長
    # top_p=1,               # Nucleus sampling，與 temperature 二選一，預設不動
    frequency_penalty=0.8,   # 減少重複字詞（0~2）
    presence_penalty=0.2,    # 鼓勵新話題（0~2）
)

# 配置嵌入模型
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    # chunk_size=64  # 每次送 16 段去嵌入
)

In [3]:
import os
import glob
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
# ✅ 改進的PDF載入函數，能夠處理損壞的PDF
def safe_load_pdfs(pdf_dir="../GPTsearch", use_directory_loader=True):
    """
    安全地載入PDF文件，跳過損壞的文件
    
    Args:
        pdf_dir: PDF文件目錄
        use_directory_loader: 是否使用DirectoryLoader (True) 或 PyPDFDirectoryLoader (False)
        
    Returns:
        loaded_documents: 成功載入的文件列表
    """
    loaded_documents = []
    failed_files = []
    
    if use_directory_loader:
        # 使用DirectoryLoader時的處理方式
        pdf_files = glob.glob(os.path.join(pdf_dir, "**", "*.pdf"), recursive=True)
        logger.info(f"找到 {len(pdf_files)} 個PDF文件")
        
        for pdf_file in pdf_files:
            try:
                logger.info(f"嘗試載入: {pdf_file}")
                loader = PyPDFLoader(pdf_file)
                docs = loader.load()
                loaded_documents.extend(docs)
                logger.info(f"成功載入: {pdf_file}, 共 {len(docs)} 頁")
            except Exception as e:
                logger.error(f"無法載入 {pdf_file}: {str(e)}")
                failed_files.append(pdf_file)
    else:
        # 使用PyPDFDirectoryLoader的替代方案
        try:
            loader = PyPDFDirectoryLoader(pdf_dir)
            loaded_documents = loader.load()
            logger.info(f"使用PyPDFDirectoryLoader載入了 {len(loaded_documents)} 個文件")
        except Exception as e:
            logger.error(f"PyPDFDirectoryLoader失敗: {str(e)}")
            # 如果整體載入失敗，改為逐個文件嘗試
            return safe_load_pdfs(pdf_dir, use_directory_loader=True)
    
    if failed_files:
        logger.warning(f"有 {len(failed_files)} 個文件無法載入: {failed_files}")
    
    logger.info(f"總共成功載入 {len(loaded_documents)} 頁PDF內容")
    return loaded_documents



In [4]:
# 載入PDF文件
documents = safe_load_pdfs("../GPTsearch")
print(f"總共載入 {len(documents)} 篇文件")

# ✅ 分段：一份 PDF 切成多個 chunk（準備送嵌入）
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200
)
splits = splitter.split_documents(documents)
print(f"總共分割成 {len(splits)} 個 chunk")

# 配置嵌入模型
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    chunk_size=64  # 每次送 16 段去嵌入
)

# ✅ 建立向量庫並保存到本地
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding_model,
    persist_directory="./chroma8_db"  # 指定本地存儲位置
)

# 確保資料被寫入磁碟
vectorstore.persist()

2025-04-18 21:15:21,014 - INFO - 找到 8 個PDF文件
2025-04-18 21:15:21,015 - INFO - 嘗試載入: ../GPTsearch\1-s2.0-S0966979519311021-main.pdf
2025-04-18 21:15:21,705 - INFO - 成功載入: ../GPTsearch\1-s2.0-S0966979519311021-main.pdf, 共 11 頁
2025-04-18 21:15:21,706 - INFO - 嘗試載入: ../GPTsearch\1-s2.0-S1003632622660862-main.pdf
2025-04-18 21:15:22,169 - INFO - 成功載入: ../GPTsearch\1-s2.0-S1003632622660862-main.pdf, 共 24 頁
2025-04-18 21:15:22,170 - INFO - 嘗試載入: ../GPTsearch\1-s2.0-S2238785422018725-main.pdf
2025-04-18 21:15:22,464 - INFO - 成功載入: ../GPTsearch\1-s2.0-S2238785422018725-main.pdf, 共 15 頁
2025-04-18 21:15:22,465 - INFO - 嘗試載入: ../GPTsearch\high-entropy-alloys-as-catalysts-for-the-co2-and-co-reduction-reactions.pdf
2025-04-18 21:15:22,603 - INFO - 成功載入: ../GPTsearch\high-entropy-alloys-as-catalysts-for-the-co2-and-co-reduction-reactions.pdf, 共 15 頁
2025-04-18 21:15:22,604 - INFO - 嘗試載入: ../GPTsearch\High‐entropy alloy catalysts_ From bulk to nano toward highly efficient carbon and nitrogen catalys

總共載入 176 篇文件
總共分割成 1023 個 chunk


2025-04-18 21:15:25,527 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-04-18 21:15:27,928 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-04-18 21:15:29,747 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-04-18 21:15:31,333 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-04-18 21:15:33,537 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-04-18 21:15:35,616 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-04-18 21:15:37,340 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-04-18 21:15:38,666 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-04-18 21:15:39,959 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK

In [8]:
# ✅ 如果要載入已存在的向量庫，可以使用以下代碼
loaded_vectorstore = Chroma(
    persist_directory="./chroma60_db",
    embedding_function=embedding_model
)

# 定義系統提示模板
# prompt = ChatPromptTemplate.from_messages([
# ("system", 
#      """You are a research assistant specializing in materials science and engineering, with expertise in literature summarization, property analysis, and application interpretation.
# Please help the user search for and organize information related to materials according to the following principles:

# - Your responses must be based on reliable sources or literature, and all sources should be clearly cited.
# - Do not fabricate data. If data is missing or unavailable, explicitly state “no data available” or “not mentioned in the literature.”
# - If multiple entries meet the query criteria, present them in a comparison table.
# - Use clear, objective, and academic language suitable for direct inclusion in technical reports.
# - Please format each summary using the following structure:
#     \n\n{context}"""),
#     MessagesPlaceholder(variable_name="history"),
#     ("human", "{question}")
# ])
# from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", 
    """You are a research assistant specializing in materials science and engineering, with expertise in literature summarization, property analysis, and application interpretation.

Your main task is to assist the user by searching, organizing, and summarizing high-entropy alloy (HEA) research according to the following principles:

- Base all responses strictly on peer-reviewed literature or credible sources, with proper citations.
- Do not fabricate any data. If data is unavailable, say “no data available” or “not mentioned in the literature.”
- When multiple relevant results are available, organize them into a comparison table.
- Use academic, objective, and concise language suitable for inclusion in technical reports or papers.
- Format your summary using the following structure:
  \n\n{context}

You may be asked to perform the following types of HEA summarization tasks. Please follow the intent behind each:

1. **By Synthesis or Processing Methods**  
   Summarize recent HEA research categorized by synthesis or processing methods (e.g., arc melting, powder metallurgy, additive manufacturing).

2. **By Major Application Categories**  
   Summarize HEA research according to major application domains (e.g., structural materials, coatings, catalysts, thermoelectrics, etc.).

3. **By Catalytic Functionality (Electrocatalysis)**  
   Summarize commonly used HEA element compositions specifically for the following catalytic reactions:  
   - Oxygen Reduction Reaction (ORR)  
   - Hydrogen Oxidation Reaction (HOR)  
   - Oxygen Evolution Reaction (OER)  
   - Hydrogen Evolution Reaction (HER)  

Be prepared to synthesize key findings, commonly used elements, and performance insights for each category."""),    
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

# 定義檢索與串接函數
def retrieve_and_concat(inputs: Dict[str, str]) -> str:
    query = inputs["question"]
    top_k = inputs.get("top_k", 5)

    # 使用動態設定的 k
    # custom_retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})
    custom_retriever = loaded_vectorstore.as_retriever(search_kwargs={"k": top_k})

    docs = custom_retriever.get_relevant_documents(query)

    context = ""
    for i, doc in enumerate(docs):
        meta = doc.metadata
        context += f"[{i+1}] 來源：{meta.get('source', '未知')}，第 {meta.get('page', '?')} 頁\n"
        context += doc.page_content + "\n\n"
    return context

retriever_chain = RunnableLambda(retrieve_and_concat)

# ✅ QA Chain 組合
qa_chain = (
    RunnableMap({
        "context": retriever_chain,
        "question": lambda x: x["question"],
        "history": lambda x: [] 
    })
    | prompt
    | generator_llm
    | StrOutputParser()
)

# 定義會話歷史獲取函數
def get_session_history(session_id: str):
    return InMemoryChatMessageHistory()

# 建立帶有記憶功能的QA鏈
qa_chain_with_memory = RunnableWithMessageHistory(
    qa_chain,                        # 原本的 QA chain
    get_session_history,            # 記憶取得函式
    input_messages_key="question",  # 問題在哪個欄位
    # output_messages_key="answer",   # 回答會存在這裡
    history_messages_key="history"  # prompt 需要的 history
)

# 提問函數
def ask_with_memory(chain, question: str, session_id: str = "default", top_k: int = 5):
    query_input = {"question": question, "top_k": top_k}
    print("🧪 問題：", question)
    response = chain.invoke(query_input, config={"configurable": {"session_id": session_id}})
    print("🤖 回答：", response)
    return response

In [12]:
# 示例查詢
ask_with_memory(qa_chain_with_memory, "請問在目前的高熵合金應用中，哪些元素組合常見於催化劑？", session_id="chen041960-2", top_k=200)



🧪 問題： 請問在目前的高熵合金應用中，哪些元素組合常見於催化劑？
🤖 回答： 在高熵合金（HEA）作為催化劑的應用中，常見的元素組合包括：

1. **氧還原反應 (ORR)**：
   - 常用組合：FeCoNiCrMn、PtPdCuAgAu
   - 這些組合通常因其多樣化的活性位點和優異的穩定性而被選擇，用於提高氧還原反應的效率。

2. **氫氧化反應 (HOR)**：
   - 常用組合：PtRu、RhPt
   - 這些元素因其在氫氣吸附和解離方面的優越性能而被廣泛使用。

3. **氧演化反應 (OER)**：
   - 常用組合：IrPdPtRhRu、CoCuGaNiZn
   - 這些高熵合金因其能夠提供多種活性位點並促進氧氣生成而受到關注。

4. **氫演化反應 (HER)**：
   - 常用組合：NiFeCoCu、PtNiFeCoCu
   - 此類組成以其出色的電催化性能和耐久性著稱，適合作為氫氣生成催化劑。

這些元素組合作為高熵合金催化劑的一部分，利用了不同金屬間協同效應來提升催化性能。[來源: Yeh et al., A perspective on the catalysis using the high-entropy alloys, Nano Energy, 2021]


'在高熵合金（HEA）作為催化劑的應用中，常見的元素組合包括：\n\n1. **氧還原反應 (ORR)**：\n   - 常用組合：FeCoNiCrMn、PtPdCuAgAu\n   - 這些組合通常因其多樣化的活性位點和優異的穩定性而被選擇，用於提高氧還原反應的效率。\n\n2. **氫氧化反應 (HOR)**：\n   - 常用組合：PtRu、RhPt\n   - 這些元素因其在氫氣吸附和解離方面的優越性能而被廣泛使用。\n\n3. **氧演化反應 (OER)**：\n   - 常用組合：IrPdPtRhRu、CoCuGaNiZn\n   - 這些高熵合金因其能夠提供多種活性位點並促進氧氣生成而受到關注。\n\n4. **氫演化反應 (HER)**：\n   - 常用組合：NiFeCoCu、PtNiFeCoCu\n   - 此類組成以其出色的電催化性能和耐久性著稱，適合作為氫氣生成催化劑。\n\n這些元素組合作為高熵合金催化劑的一部分，利用了不同金屬間協同效應來提升催化性能。[來源: Yeh et al., A perspective on the catalysis using the high-entropy alloys, Nano Energy, 2021]'

In [13]:
ask_with_memory(qa_chain_with_memory, "哪一組合金性能最好？", session_id="chen041960-2", top_k=50)


🧪 問題： 哪一組合金性能最好？
🤖 回答： 要比較高熵合金（HEA）的性能，首先需要明確所關心的性能指標，例如機械強度、耐磨性、抗腐蝕性等。以下是一些常見的高熵合金及其相關性能數據：

| 合金組成 | 屈服強度 (MPa) | 極限抗拉強度 (UTS, MPa) | 延伸率 (%) | 硬度 (HV) |
|---------|-----------------|--------------------------|------------|-----------|
| AlCoCrCuFeNi [2] | 不詳 | 不詳 | 不詳 | 120-650（隨Al含量增加） |
| HfNbTaTiZr [5]  | 不詳  | 不詳   | 不詳  | 高硬度和良好的耐磨性 |
| Al0.5CrFe1.5MnNi0.5 [30]  | 高於高速鋼的軟化阻力和良好的耐磨性 |

根據不同文獻中的數據，可以看到不同合金在特定應用或條件下表現出優異的性能。例如，AlCoCrCuFeNi 隨著鋁含量增加，其硬度顯著提高，這使得它在結構和工具行業中具有潛力[2]。而HfNbTaTiZr則因其高硬度和良好的耐磨性被認為適合極端環境[5]。

具體哪一種合金性能最好取決於應用需求。如果您能提供更詳細的需求，我可以幫助進一步篩選或比較。


'要比較高熵合金（HEA）的性能，首先需要明確所關心的性能指標，例如機械強度、耐磨性、抗腐蝕性等。以下是一些常見的高熵合金及其相關性能數據：\n\n| 合金組成 | 屈服強度 (MPa) | 極限抗拉強度 (UTS, MPa) | 延伸率 (%) | 硬度 (HV) |\n|---------|-----------------|--------------------------|------------|-----------|\n| AlCoCrCuFeNi [2] | 不詳 | 不詳 | 不詳 | 120-650（隨Al含量增加） |\n| HfNbTaTiZr [5]  | 不詳  | 不詳   | 不詳  | 高硬度和良好的耐磨性 |\n| Al0.5CrFe1.5MnNi0.5 [30]  | 高於高速鋼的軟化阻力和良好的耐磨性 |\n\n根據不同文獻中的數據，可以看到不同合金在特定應用或條件下表現出優異的性能。例如，AlCoCrCuFeNi 隨著鋁含量增加，其硬度顯著提高，這使得它在結構和工具行業中具有潛力[2]。而HfNbTaTiZr則因其高硬度和良好的耐磨性被認為適合極端環境[5]。\n\n具體哪一種合金性能最好取決於應用需求。如果您能提供更詳細的需求，我可以幫助進一步篩選或比較。'

In [20]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.vectorstores import Chroma
# from qdrant_client import QdrantClient
# from langchain_qdrant import QdrantVectorStore
from langchain_core.runnables.history import RunnableWithMessageHistory
from typing import Dict, List, Tuple
from langchain.schema.runnable import RunnableMap, RunnableLambda
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain.schema.document import Document
import os
import time
from datetime import datetime
import pandas as pd

# 配置 LLM 模型
generator_llm = ChatOpenAI(
    model_name="gpt-4o",   # 或 "gpt-4", "gpt-3.5-turbo"
    temperature=0.3,       # 控制回答穩定性。推薦值：0~0.3（任務導向），0.7~1（創意任務）
    max_tokens=8192,       # 控制回應長度。推薦根據任務調整，如摘要可短，技術問答可設長
    # top_p=1,               # Nucleus sampling，與 temperature 二選一，預設不動
    frequency_penalty=0.8,   # 減少重複字詞（0~2）
    presence_penalty=0.2,    # 鼓勵新話題（0~2）
)

# 配置嵌入模型
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    chunk_size=64  # 每次送 64 段去嵌入
)

# ✅ 如果要載入已存在的向量庫，可以使用以下代碼
vectorstore = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embedding_model
)

# 獲取向量庫的總文檔數量
total_documents = vectorstore._collection.count()
print(f"向量庫中共有 {total_documents} 個文檔片段")

# 定義系統提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """You are a research assistant specializing in materials science and engineering, with expertise in literature summarization, property analysis, and application interpretation.
Please help the user search for and organize information related to materials according to the following principles:

- Your responses must be based on reliable sources or literature, and all sources should be clearly cited.
- Do not fabricate data. If data is missing or unavailable, explicitly state "no data available" or "not mentioned in the literature."
- If multiple entries meet the query criteria, present them in a comparison table.
- Use clear, objective, and academic language suitable for direct inclusion in technical reports.
- Please format each summary using the following structure:
    \n\n{context}"""),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

# 創建檢索結果的詳細資訊結構
class RetrievalInfo:
    def __init__(self):
        self.total_docs_retrieved = 0
        self.unique_sources = set()
        self.retrieval_time = 0
        self.doc_details = []
        self.similarity_scores = []
    
    def add_document(self, doc: Document, score: float = None):
        self.total_docs_retrieved += 1
        source = doc.metadata.get('source', '未知')
        self.unique_sources.add(source)
        
        # 添加文檔詳細信息
        self.doc_details.append({
            'source': source,
            'page': doc.metadata.get('page', '?'),
            'content_preview': doc.page_content[:100] + '...' if len(doc.page_content) > 100 else doc.page_content,
            'content_length': len(doc.page_content)
        })
        
        if score is not None:
            self.similarity_scores.append(score)
    
    def get_summary(self) -> str:
        summary = []
        summary.append(f"📊 檢索統計:")
        summary.append(f"- 共檢索到 {self.total_docs_retrieved} 個文檔片段")
        summary.append(f"- 來自 {len(self.unique_sources)} 個不同來源")
        summary.append(f"- 檢索耗時: {self.retrieval_time:.2f} 秒")
        
        if self.similarity_scores:
            summary.append(f"- 相似度分數範圍: {min(self.similarity_scores):.3f} - {max(self.similarity_scores):.3f}")
        
        # 創建文檔詳情表格（不使用tabulate）
        if self.doc_details:
            summary.append("\n📑 檢索文檔詳情:")
            # 添加表頭
            summary.append("索引 | 來源 | 頁碼 | 內容長度 | 內容預覽")
            summary.append("----- | ----- | ----- | ----- | -----")
            
            # 添加數據行
            for i, doc in enumerate(self.doc_details):
                summary.append(f"{i} | {doc['source']} | {doc['page']} | {doc['content_length']} | {doc['content_preview']}")
        
        return "\n".join(summary)

# 全局變數，用於存儲最近一次檢索的詳細信息
last_retrieval_info = RetrievalInfo()

# 定義檢索與串接函數，加入檢索統計功能
def retrieve_and_concat(inputs: Dict[str, str]) -> str:
    global last_retrieval_info
    
    query = inputs["question"]
    top_k = inputs.get("top_k", 5)
    
    retrieval_info = RetrievalInfo()
    
    print(f"\n⏳ 正在檢索與「{query}」相關的文檔，檢索數量: {top_k}")
    start_time = time.time()
    
    # 使用動態設定的 k
    try:
        # 首先嘗試使用帶分數的檢索方法
        custom_retriever = vectorstore.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={"k": top_k, "score_threshold": 0.0}  # 設置閾值為0確保返回所有結果
        )
        docs_and_scores = custom_retriever.get_relevant_documents_and_scores(query)
        
        # 處理檢索結果
        context = ""
        for i, (doc, score) in enumerate(docs_and_scores):
            retrieval_info.add_document(doc, score)
            meta = doc.metadata
            context += f"[{i+1}] 來源：{meta.get('source', '未知')}，第 {meta.get('page', '?')} 頁 (相似度: {score:.3f})\n"
            context += doc.page_content + "\n\n"
    
    except (AttributeError, NotImplementedError):
        # 如果上面的方法失敗，回退到普通檢索
        print("⚠️ 帶分數檢索方法不可用，回退到普通檢索")
        custom_retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})
        docs = custom_retriever.get_relevant_documents(query)
        
        # 處理檢索結果
        context = ""
        for i, doc in enumerate(docs):
            retrieval_info.add_document(doc)
            meta = doc.metadata
            context += f"[{i+1}] 來源：{meta.get('source', '未知')}，第 {meta.get('page', '?')} 頁\n"
            context += doc.page_content + "\n\n"
    
    retrieval_info.retrieval_time = time.time() - start_time
    last_retrieval_info = retrieval_info
    
    # 打印檢索統計信息
    print(retrieval_info.get_summary())
    
    return context

retriever_chain = RunnableLambda(retrieve_and_concat)

# ✅ QA Chain 組合
qa_chain = (
    RunnableMap({
        "context": retriever_chain,
        "question": lambda x: x["question"],
        "history": lambda x: [] 
    })
    | prompt
    | generator_llm
    | StrOutputParser()
)

# 定義會話歷史獲取函數
def get_session_history(session_id: str):
    return InMemoryChatMessageHistory()

# 建立帶有記憶功能的QA鏈
qa_chain_with_memory = RunnableWithMessageHistory(
    qa_chain,                        # 原本的 QA chain
    get_session_history,            # 記憶取得函式
    input_messages_key="question",  # 問題在哪個欄位
    # output_messages_key="answer",   # 回答會存在這裡
    history_messages_key="history"  # prompt 需要的 history
)

# 提問函數，添加查詢資訊記錄
def ask_with_memory(chain, question: str, session_id: str = "default", top_k: int = 5, show_retrieval_info: bool = True):
    # 記錄查詢開始時間
    start_time = time.time()
    print(f"\n===== {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} =====")
    print(f"🧪 問題：{question}")
    print(f"🔍 會話ID: {session_id}, 檢索數量: {top_k}")
    
    # 執行查詢
    query_input = {"question": question, "top_k": top_k}
    response = chain.invoke(query_input, config={"configurable": {"session_id": session_id}})
    
    # 記錄總耗時
    total_time = time.time() - start_time
    
    print(f"\n🤖 回答：{response}")
    print(f"\n⏱️ 總耗時: {total_time:.2f} 秒")
    
    # 保存查詢記錄到文件
    with open(f"query_log_{session_id}.txt", "a", encoding="utf-8") as f:
        f.write(f"===== {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} =====\n")
        f.write(f"問題: {question}\n")
        f.write(f"檢索數量: {top_k}\n")
        f.write(f"檢索文檔數: {last_retrieval_info.total_docs_retrieved}\n")
        f.write(f"回答: {response}\n")
        f.write(f"總耗時: {total_time:.2f} 秒\n\n")
    
    return response

# 獲取最近一次檢索的詳細信息
def get_last_retrieval_info():
    global last_retrieval_info
    return last_retrieval_info.get_summary()

# ✅ 向量庫統計功能
def get_vectorstore_stats():
    """獲取向量庫的統計信息"""
    total_docs = vectorstore._collection.count()
    
    # 嘗試獲取所有文檔的元數據
    try:
        metadata_list = vectorstore._collection.get(include=["metadatas"])["metadatas"]
        
        # 分析來源和頁數
        sources = {}
        page_counts = {}
        
        for metadata in metadata_list:
            source = metadata.get('source', '未知')
            page = metadata.get('page', '未知')
            
            if source in sources:
                sources[source] += 1
            else:
                sources[source] = 1
                
            if page in page_counts:
                page_counts[page] += 1
            else:
                page_counts[page] = 1
        
        # 生成統計報告
        stats = []
        stats.append(f"📚 向量庫統計報告:")
        stats.append(f"- 總文檔數: {total_docs}")
        stats.append(f"- 唯一來源數: {len(sources)}")
        
        # 來源分布
        stats.append("\n📂 文檔來源分布:")
        for source, count in sorted(sources.items(), key=lambda x: x[1], reverse=True):
            stats.append(f"- {source}: {count} 個片段 ({count/total_docs*100:.1f}%)")
        
        return "\n".join(stats)
        
    except Exception as e:
        return f"📚 向量庫統計報告:\n- 總文檔數: {total_docs}\n- 詳細統計獲取失敗: {str(e)}"

# 打印向量庫統計信息
print(get_vectorstore_stats())

# 示例查詢
print("\n======== 開始查詢示例 ========")
ask_with_memory(qa_chain_with_memory, "請問在目前的高熵合金應用中，哪些元素組合常見於催化劑？", session_id="chen03", top_k=100)
print("\n---- 最近一次檢索的詳細信息 ----")
print(get_last_retrieval_info())

ask_with_memory(qa_chain_with_memory, "哪一組合金性能最好？", session_id="chen03", top_k=20)
print("\n---- 最近一次檢索的詳細信息 ----")
print(get_last_retrieval_info())

向量庫中共有 81427 個文檔片段
📚 向量庫統計報告:
- 總文檔數: 81427
- 唯一來源數: 1090

📂 文檔來源分布:
- ../PDF\Zhang …等 - 2024 - Frontiers in high entropy alloys and high entropy functional materials.pdf: 1097 個片段 (1.3%)
- ../PDF\Additive-manufacturing-of-nickel-based-superalloys--A-sta_2023_Progress-in-M.pdf: 909 個片段 (1.1%)
- ../PDF\Cantor - 2021 - Multicomponent high-entropy Cantor alloys.pdf: 527 個片段 (0.6%)
- ../PDF\Mishra …等 - 2021 - High entropy alloys  Tunability of deformation mechanisms through integration of compositional and.pdf: 489 個片段 (0.6%)
- ../PDF\Li …等 - 2019 - Mechanical properties of high-entropy alloys with emphasis on face-centered cubic alloys.pdf: 357 個片段 (0.4%)
- ../PDF\Liu …等 - 2021 - The Evolution of Intermetallic Compounds in High-Entropy Alloys From the Secondary Phase to the Mai.pdf: 321 個片段 (0.4%)
- ../PDF\Mooraj和Chen - 2023 - A review on high-throughput development of high-entropy alloys by combinatorial methods.pdf: 284 個片段 (0.3%)
- ../PDF\Rogachev - 2020 - Structure, Stability, and Pro

2025-04-12 16:00:26,026 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


📊 檢索統計:
- 共檢索到 100 個文檔片段
- 來自 22 個不同來源
- 檢索耗時: 1.29 秒

📑 檢索文檔詳情:
索引 | 來源 | 頁碼 | 內容長度 | 內容預覽
----- | ----- | ----- | ----- | -----
0 | ../PDF\Zhang …等 - 2022 - High-Entropy Alloys for Electrocatalysis Design, Characterization, and Applications.pdf | 0 | 731 | metals (e.g., Fe, Co, Ni, and Cu); thus, these catalysts generally comprise no more than three eleme...
1 | ../PDF\A-perspective-on-the-catalysis-using-the-high-entropy-alloys_2021_Nano-Energ.pdf | 1 | 781 | material. Therefore, many combinations by picking elements from both 
the groups (depicted in Fig. 1...
2 | ../PDF\Katiyar …等 - 2021 - A perspective on the catalysis using the high entropy alloys.pdf | 1 | 781 | material. Therefore, many combinations by picking elements from both 
the groups (depicted in Fig. 1...
3 | ../PDF\High‐entropy alloy catalysts_ From bulk to nano toward highly efficient carbon and nitrogen catalysis - Yu - 2022 - Carbon Energy - Wiley Online Library.pdf | 22 | 721 | neutral and acidic electrolytes. In 

2025-04-12 16:00:36,900 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🤖 回答：在高熵合金（HEAs）催化劑的應用中，常見的元素組合主要包括過渡金屬和一些貴金屬。以下是一些常見的元素組合：

1. **過渡金屬**：這些元素通常包括鐵（Fe）、鈷（Co）、鎳（Ni）、銅（Cu）、鉻（Cr）、錳（Mn）等。這些元素因其豐富的儲量和良好的電催化性能而被廣泛使用。[6]

2. **貴金屬**：如鉑（Pt）、鈀（Pd）、銠（Rh）、釕（Ru）等，這些元素通常與過渡金屬結合以提高催化性能。[13]

3. **多元組合**：例如AgAuCuPdPt、PtPdCuAgAu等多元高熵合金，這些組合展示了優異的氧化還原能力和穩定性。[2][49]

4. **非貴重金屬替代**：有研究開發出由低成本、非貴重金屬如鋁(Al)、鋅(Zn)等構成的HEAs，用於降低成本並提升可持續性。[15][59]

5. **特殊應用中的特定組合**：例如，在氮氣還原反應(NRR)中，有效利用了鐵(Fe)、鈷(Co)、鎳(Ni)及其與其他元素如釕(Ru)或銅(Cu)的協同作用來增強催化活性。[4]

以上信息來源於文獻中的不同研究報告，其中提到了各種高熵合金在不同反應中的潛力和實際應用效果。

⏱️ 總耗時: 11.48 秒

---- 最近一次檢索的詳細信息 ----
📊 檢索統計:
- 共檢索到 100 個文檔片段
- 來自 22 個不同來源
- 檢索耗時: 1.29 秒

📑 檢索文檔詳情:
索引 | 來源 | 頁碼 | 內容長度 | 內容預覽
----- | ----- | ----- | ----- | -----
0 | ../PDF\Zhang …等 - 2022 - High-Entropy Alloys for Electrocatalysis Design, Characterization, and Applications.pdf | 0 | 731 | metals (e.g., Fe, Co, Ni, and Cu); thus, these catalysts generally comprise no more than three eleme...
1 | ../PDF\A-perspective-on-the-catalysis-using-the-high-entropy-alloys_2021_Nano-Energ.pdf 

2025-04-12 16:00:37,727 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


📊 檢索統計:
- 共檢索到 20 個文檔片段
- 來自 12 個不同來源
- 檢索耗時: 1.51 秒

📑 檢索文檔詳情:
索引 | 來源 | 頁碼 | 內容長度 | 內容預覽
----- | ----- | ----- | ----- | -----
0 | ../PDF\Xu …等 - 2022 - Toughening Pathways and Regulatory Mechanisms of Refractory High-Entropy Alloys.pdf | 4 | 759 | 力学性能 。Huang等[36]设计的 TaxHfZrTi双相难熔高
熵合金表现出较大的延伸率 ，由于 TRIP效应诱导
hcp相形成，合金中 bcc相和 hcp相之间的动态应变 -
应力分配产生强烈...
1 | ../PDF\Zhao …等 - 2024 - Research Status of Phase Transformation Behavior of High Entropy Shape Memory Alloys.pdf | 3 | 668 | 当Cr含量为 10%（质量分数）时合金性能较好。朱琳[44]
研究了 AlCoCrFeNi x在热处理过程中的相变行为，如图 3
所示。当 x=0.4，0.8时，在 600 ℃发生单一的可逆相变。
当...
2 | ../PDF\Xu …等 - 2022 - Toughening Pathways and Regulatory Mechanisms of Refractory High-Entropy Alloys.pdf | 7 | 410 | 开发具有高强高韧的金属材料一直是材料领域
的研究热门 。传统金属材料常采用细晶强化 、固溶
强化、第二相强化和相变强化等强韧化手段达到改
善力学性能的目的 ，就目前研究而言 ，只有绝大部分
3d过渡金...
3 | ../PDF\Liu …等 - 2023 - Progress of Cryogenic Deformation and Strengthening-Toughening Mechanisms of High-Entropy Alloys.pdf | 9 | 799 | 59 卷 金 属 学 报
超高强度水平 ，同时还保持 53%的优异延伸率 ，如
图10b[9

2025-04-12 16:00:49,587 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🤖 回答：要确定哪一组合金性能最好，我们需要明确评价标准，例如强度、韧性、耐腐蚀性等。以下是一些高熵合金的性能比较：

| 合金类型 | 屈服强度 (MPa) | 抗拉强度 (MPa) | 延伸率 (%) | 其他特性 |
|----------|-----------------|-----------------|------------|----------|
| FeCoNiCrMn [6] | 563 MPa | 未提及 | 52% | 单相 fcc 结构，良好的塑性 |
| TiZrVNb [1] | 高强低韧（具体数值未提及） | 未提及 | 未提及 | V2Zr 相析出强化，但破坏了韧性 |
| AlCoCrFeNi2.1 [14] | 屈服强度未提及，抗拉强度超过1 GPa, 延伸率39%[12]，700 MPa (77 K)[4]，断裂强度超过1 GPa[4], 延伸率9%[4]| 优异的低温性能，共晶组织由 B2 和 fcc 相组成 |

从上表可以看出，不同合金在不同条件下表现出不同的优异性能。例如：
- FeCoNiCrMn 合金具有良好的延展性和单相结构。
- AlCoCrFeNi2.1 共晶高熵合金在低温下表现出优异的力学性能，并且其共晶组织提供了良好的综合力学性能。

因此，“最好”的定义取决于应用场景和所需特定属性。如果需要更详细的信息或针对特定应用场景，请提供更多细节以便进一步分析。

⏱️ 總耗時: 12.69 秒

---- 最近一次檢索的詳細信息 ----
📊 檢索統計:
- 共檢索到 20 個文檔片段
- 來自 12 個不同來源
- 檢索耗時: 1.51 秒

📑 檢索文檔詳情:
索引 | 來源 | 頁碼 | 內容長度 | 內容預覽
----- | ----- | ----- | ----- | -----
0 | ../PDF\Xu …等 - 2022 - Toughening Pathways and Regulatory Mechanisms of Refractory High-Entropy Alloys.pdf | 4 | 759 | 力学性能 。Huang等[36]设计的 TaxHfZrTi双相难熔高
熵合金表现出较大的延伸率 ，由于 TRIP效应诱导
hcp相形成，合金中 bcc相和 hcp相之间的动态应